#### *Instalando as bibliotecas que serão necessárias*

In [2]:
# Instala a biblioteca "pandas" que será usada para manipulação de dados.
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Instala a biblioteca "numpy" que será usada para operações numéricas.
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Instala a biblioteca "yfinance" que será usada para obter dados financeiros.
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [5]:
# Instala a biblioteca "plotly" que será usada para a exibição de gráficos. 
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable


#### *Importando as bibliotecas que serão necessárias*

In [ ]:
# Importa a biblioteca "pandas", que será usada para manipulação de dados.
import pandas as pd
# Importa a biblioteca "numpy", que será usada para operações numéricas.
import numpy as np
# Importa a biblioteca "yfinance" para obter alguns dados de certas ações.
import yfinance as yf
# Importa o módulo "graph_objects" da biblioteca "plotly" para lidar com a exibição de gráficos.
import plotly.graph_objects as go
# Importa o módulo datetime da biblioteca "datetime" para lidar com datas.
from datetime import datetime, timedelta
# Importa a biblioteca "math" para lidar com certas operações matemáticas.
import math
# Importa o módulo Optional da biblioteca "typing" para lidar com a padronização dos tipos de parâmetros opcionais das funções.
from typing import Optional

# Código do backtest

PRIMEIRO TESTE PARA O BACKTEST
Cálculo do S&P ta errado

In [ ]:
# Valor inicial do portfólio
valor_portfolio_inicial = 100000

# Listas para armazenar o histórico do portfólio e as datas correspondentes
historico_portfolio = []
datas_portfolio = []

# Ler todas as planilhas do arquivo Excel
sheets = pd.read_excel('resultados_portfolio_mensal_full_modified.xlsx', sheet_name=None, header=0)

for nome_sheet, df in sheets.items():
    # Verificar se o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' está vazia. Pulando este período.")
        continue

    # Verificar se as colunas esperadas estão presentes
    colunas_esperadas = ['Acao', 'Retorno Predito', 'Retorno Real']
    if not all(col in df.columns for col in colunas_esperadas):
        print(f"A planilha '{nome_sheet}' não contém as colunas esperadas. Pulando este período.")
        continue

    # Extrair o ticker, data de início e data de fim da coluna 'Acao'
    try:
        # Aplicar a extração em cada linha
        def extrair_dados(row):
            acao_periodo = row['Acao']
            if pd.isnull(acao_periodo):
                return pd.Series([np.nan, np.nan, np.nan])
            parts = acao_periodo.split(':')
            simbolo = parts[0].strip()
            datas = parts[1].strip().split(' - ')
            data_inicio = pd.to_datetime(datas[0].strip(), format='%Y-%m-%d')
            data_fim = pd.to_datetime(datas[1].strip(), format='%Y-%m-%d')
            return pd.Series([simbolo, data_inicio, data_fim])

        df[['Simbolo', 'Data_Inicio', 'Data_Fim']] = df.apply(extrair_dados, axis=1)
        # Remover linhas com dados ausentes
        df.dropna(subset=['Simbolo', 'Data_Inicio', 'Data_Fim'], inplace=True)
    except Exception as e:
        print(f"Erro ao extrair os dados da planilha '{nome_sheet}': {e}")
        continue

    # Obter a data de início e fim do período a partir da primeira linha
    data_inicio = df['Data_Inicio'].iloc[0]
    data_fim = df['Data_Fim'].iloc[0]

    # Ordenar pelo retorno predito
    df = df.sort_values(by='Retorno Predito', ascending=False)

    # Selecionar top 10%
    n_acoes = max(int(len(df) * 0.10), 1)
    acoes_compradas = df.head(n_acoes)

    # Alocar capital igualmente entre as ações
    valor_portfolio = valor_portfolio_inicial if not historico_portfolio else historico_portfolio[-1]
    alocacao_por_acao = valor_portfolio / n_acoes

    # Calcular o retorno real
    retornos = acoes_compradas['Retorno Real'] / 100  # Converter porcentagem para decimal
    valor_final = alocacao_por_acao * (1 + retornos)

    # Atualizar o valor do portfólio
    valor_portfolio = valor_final.sum()
    historico_portfolio.append(valor_portfolio)
    datas_portfolio.append(data_fim)

    # Imprimir os resultados do período
    print(f"Período: {data_inicio.date()} - {data_fim.date()}")
    print(f"Ações compradas: {acoes_compradas['Simbolo'].tolist()}")
    print("Retorno de cada ação:")
    for index, row in acoes_compradas.iterrows():
        print(f" - {row['Simbolo']}: {row['Retorno Real']}%")
    print(f"Valor do portfólio ao final do período: ${valor_portfolio:.2f}\n")

# Verificar se o histórico do portfólio está vazio
if not historico_portfolio or not datas_portfolio:
    print("Nenhum período foi processado com sucesso. Verifique os dados e tente novamente.")
else:
    # Criar DataFrame com o histórico do portfólio
    df_portfolio = pd.DataFrame({
        'Data': datas_portfolio,
        'Valor': historico_portfolio
    })
    # Ordenar por data
    df_portfolio.sort_values(by='Data', inplace=True)
    df_portfolio.reset_index(drop=True, inplace=True)

    # 5. Obter dados do S&P500
    data_inicio_total = df_portfolio['Data'].iloc[0] - timedelta(days=5)
    data_fim_total = df_portfolio['Data'].iloc[-1] + timedelta(days=5)
    sp500 = yf.download('^GSPC', start=data_inicio_total, end=data_fim_total)
    print("Colunas disponíveis em sp500:", sp500.columns)

    # Determinar a coluna de preço a ser usada
    if 'Adj Close' in sp500.columns:
        price_column = 'Adj Close'
    elif 'Close' in sp500.columns:
        price_column = 'Close'
    else:
        raise KeyError("Nenhuma coluna de preço ('Adj Close' ou 'Close') encontrada em sp500")

    # Resetar o índice para ter a coluna 'Date'
    sp500.reset_index(inplace=True)

    # Remover informações de fuso horário da coluna 'Date'
    sp500['Date'] = sp500['Date'].dt.tz_localize(None)

    # Verificar os tipos de dados das colunas de datas
    print("dtype de sp500['Date']:", sp500['Date'].dtype)
    print("dtype de df_portfolio['Data']:", df_portfolio['Data'].dtype)

    # Calcular retorno acumulado do S&P500
    sp500['Retorno'] = sp500[price_column].pct_change().fillna(0)
    sp500['Retorno Acumulado'] = (1 + sp500['Retorno']).cumprod()

    # Interpolar o retorno acumulado do S&P500 nas datas do portfólio
    sp500_interp = sp500[['Date', 'Retorno Acumulado']].set_index('Date').reindex(
        df_portfolio['Data'], method='ffill')

    # Após reindexar, resetamos o índice
    sp500_interp.reset_index(inplace=True)

    # Verificar as colunas e as primeiras linhas
    print("Colunas de sp500_interp após reset_index:", sp500_interp.columns)
    print("Primeiras linhas de sp500_interp:")
    print(sp500_interp.head())

    # Ajustar o acesso à coluna de datas de sp500_interp
    if 'Date' in sp500_interp.columns:
        date_column = 'Date'
    elif 'index' in sp500_interp.columns:
        date_column = 'index'
    elif 'Data' in sp500_interp.columns:
        date_column = 'Data'
    else:
        print("A coluna de datas não foi encontrada em sp500_interp.")
        print("Colunas disponíveis:", sp500_interp.columns)
        raise KeyError("Coluna de datas não encontrada em sp500_interp")

    # Calcular retorno acumulado da estratégia
    df_portfolio['Retorno'] = df_portfolio['Valor'].pct_change().fillna(0)
    df_portfolio['Retorno Acumulado'] = (1 + df_portfolio['Retorno']).cumprod()

    # 6. Plotar o gráfico comparativo usando Plotly
    trace_estrategia = go.Scatter(
        x=df_portfolio['Data'],
        y=df_portfolio['Retorno Acumulado'],
        mode='lines+markers',
        name='Retorno Acumulado da Estratégia',
        yaxis='y1'
    )

    trace_sp500 = go.Scatter(
        x=sp500_interp[date_column],
        y=sp500_interp['Retorno Acumulado'],
        mode='lines+markers',
        name='Retorno Acumulado do S&P500',
        yaxis='y1'
    )

    trace_capital = go.Scatter(
        x=df_portfolio['Data'],
        y=df_portfolio['Valor'],
        mode='lines+markers',
        name='Capital do Portfólio',
        yaxis='y2'
    )

    data = [trace_estrategia, trace_sp500, trace_capital]

    layout = go.Layout(
        title='Comparação de Retorno Acumulado e Capital do Portfólio',
        xaxis=dict(title='Data'),
        yaxis=dict(
            title='Retorno Acumulado',
            side='left',
            showgrid=False,
            zeroline=False
        ),
        yaxis2=dict(
            title='Capital do Portfólio',
            overlaying='y',
            side='right',
            showgrid=False,
            zeroline=False
        ),
        legend=dict(x=0.01, y=0.99)
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

# 7. Plotar o histograma da distribuição dos retornos da estratégia

# Remover possíveis NaNs ou valores infinitos
retornos = df_portfolio['Retorno'].dropna().replace([np.inf, -np.inf], np.nan).dropna()

# Criar o histograma dos retornos
histograma = go.Figure(data=[go.Histogram(
    x=retornos * 100,  # Converter para porcentagem
    xbins=dict(
        start=retornos.min() * 100,
        end=retornos.max() * 100,
        size=1  # Intervalo de 1% entre as barras
    ),
    marker_color='blue',
    opacity=0.75
)])

histograma.update_layout(
    title='Distribuição dos Retornos da Estratégia',
    xaxis_title='Retorno (%)',
    yaxis_title='Frequência',
    bargap=0.2,
    bargroupgap=0.1
)

Período: 2019-02-22 - 2019-03-21
Ações compradas: ['AMT', 'SO', 'MCD', 'WMT', 'XEL']
Retorno de cada ação:
 - AMT: 11.70644120100242%
 - SO: 4.33944426916547%
 - MCD: 2.262881295990945%
 - WMT: 1.500255889162151%
 - XEL: 4.488114776519038%
Valor do portfólio ao final do período: $104859.43

Período: 2019-03-22 - 2019-04-18
Ações compradas: ['WRB', 'SYY', 'WMT', 'L', 'YUM']
Retorno de cada ação:
 - WRB: 3.387195385279303%
 - SYY: 8.42385272730434%
 - WMT: 5.960145210909561%
 - L: 2.258000103301393%
 - YUM: 4.291298434993405%
Valor do portfólio ao final do período: $109959.89

Período: 2019-04-22 - 2019-05-17
Ações compradas: ['SBAC', 'AFL', 'SYY', 'TRV', 'AMT']
Retorno de cada ação:
 - SBAC: 5.104769925604463%
 - AFL: 5.921211468510501%
 - SYY: 7.709974903555966%
 - TRV: 5.792061200895203%
 - AMT: 5.138936845215272%
Valor do portfólio ao final do período: $116484.24

Período: 2019-05-20 - 2019-06-17
Ações compradas: ['PPL', 'MDLZ', 'WRB', 'ALL', 'YUM']
Retorno de cada ação:
 - PPL: 3.21

[*********************100%***********************]  1 of 1 completed

Período: 2022-04-26 - 2022-05-23
Ações compradas: ['HOLX', 'GILD', 'NI', 'KMI', 'PNW']
Retorno de cada ação:
 - HOLX: 9.051267057995403%
 - GILD: 7.027302884126526%
 - NI: 5.446473265168865%
 - KMI: 5.415419361546443%
 - PNW: 5.452546782698813%
Valor do portfólio ao final do período: $1099366.06

Período: 2022-05-24 - 2022-06-22
Ações compradas: ['NDAQ', 'CBOE', 'IBM', 'K', 'HII']
Retorno de cada ação:
 - NDAQ: -0.9222760468970544%
 - CBOE: -2.233061398944542%
 - IBM: -1.585003305208495%
 - K: -1.910382958613686%
 - HII: -2.335836710470279%
Valor do portfólio ao final do período: $1079607.02

Período: 2022-06-23 - 2022-07-21
Ações compradas: ['HD', 'VICI', 'CBOE', 'WMT', 'VRSK']
Retorno de cada ação:
 - HD: 12.13344957732011%
 - VICI: 7.847793077436845%
 - CBOE: 8.346929275840708%
 - WMT: 7.991087168387426%
 - VRSK: 8.924345484247887%
Valor do portfólio ao final do período: $1177297.65

Período: 2022-07-22 - 2022-08-18
Ações compradas: ['AFL', 'TJX', 'ALL', 'WRB', 'ECL']
Retorno de cad

TESTE 2 pq o primeiro tava dando problema com o S&P500

Tudo fragmentado pra facilitar identificação de erro

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import timedelta
import plotly.graph_objs as go

# Valor inicial do portfólio
valor_portfolio_inicial = 100000

# Listas para armazenar o histórico do portfólio e as datas correspondentes
historico_portfolio = []
datas_portfolio = []
datas_inicio_portfolio = []
retornos_estrategia_periodo = []
retornos_sp500_periodo = []

# Nome do arquivo Excel
arquivo_excel = 'resultados_portfolio_mensal_full_modified.xlsx'

# Tentar ler o arquivo Excel
try:
    sheets = pd.read_excel(arquivo_excel, sheet_name=None, header=0)
    print(f"Arquivo '{arquivo_excel}' lido com sucesso.")
except FileNotFoundError:
    print(f"Arquivo '{arquivo_excel}' não encontrado. Verifique o caminho e o nome do arquivo.")
    exit()

# Função para extrair dados de cada linha
def extrair_dados(row):
    acao_periodo = row['Acao']
    if pd.isnull(acao_periodo):
        return pd.Series([np.nan, np.nan, np.nan])
    parts = acao_periodo.split(':')
    if len(parts) != 2:
        return pd.Series([np.nan, np.nan, np.nan])
    simbolo = parts[0].strip()
    datas = parts[1].strip().split(' - ')
    if len(datas) != 2:
        return pd.Series([np.nan, np.nan, np.nan])
    try:
        data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
        data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
    except ValueError:
        return pd.Series([np.nan, np.nan, np.nan])
    return pd.Series([simbolo, data_inicio, data_fim])

# Processar cada planilha
for nome_sheet, df in sheets.items():
    print(f"\nProcessando a planilha: {nome_sheet}")
    
    # Verificar se o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' está vazia. Pulando este período.")
        continue

    # Verificar se as colunas esperadas estão presentes
    colunas_esperadas = ['Acao', 'Retorno Predito', 'Retorno Real']
    if not all(col in df.columns for col in colunas_esperadas):
        print(f"A planilha '{nome_sheet}' não contém as colunas esperadas: {colunas_esperadas}. Pulando este período.")
        continue

    # Aplicar a extração em cada linha
    df[['Simbolo', 'Data_Inicio', 'Data_Fim']] = df.apply(extrair_dados, axis=1)

    # Remover linhas com dados ausentes
    linhas_antes = len(df)
    df.dropna(subset=['Simbolo', 'Data_Inicio', 'Data_Fim'], inplace=True)
    linhas_depois = len(df)
    print(f"Linhas processadas: {linhas_depois} (removidas {linhas_antes - linhas_depois} linhas com dados ausentes)")

    # Verificar se após a extração o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' não possui dados válidos após a extração. Pulando este período.")
        continue

    # Obter a data de início e fim do período a partir da primeira linha
    data_inicio = df['Data_Inicio'].iloc[0]
    data_fim = df['Data_Fim'].iloc[0]

    print(f"Período: {data_inicio.date()} - {data_fim.date()}")

    # Ordenar pelo retorno predito
    df = df.sort_values(by='Retorno Predito', ascending=False)

    # Selecionar top 10%
    n_acoes = max(int(len(df) * 0.10), 1)
    acoes_compradas = df.head(n_acoes)
    print(f"Selecionadas {n_acoes} ações para o portfólio.")

    # Alocar capital igualmente entre as ações
    if not historico_portfolio:
        valor_portfolio_anterior = valor_portfolio_inicial
    else:
        valor_portfolio_anterior = historico_portfolio[-1]
    valor_portfolio = valor_portfolio_anterior
    alocacao_por_acao = valor_portfolio / n_acoes

    # Calcular o retorno real
    retornos = acoes_compradas['Retorno Real'] / 100  # Converter porcentagem para decimal
    valor_final = alocacao_por_acao * (1 + retornos)

    # Atualizar o valor do portfólio
    valor_portfolio = valor_final.sum()
    historico_portfolio.append(valor_portfolio)
    datas_portfolio.append(data_fim)
    datas_inicio_portfolio.append(data_inicio)

    # Calcular o retorno do período
    retorno_periodo = (valor_portfolio - valor_portfolio_anterior) / valor_portfolio_anterior * 100  # Em porcentagem
    retornos_estrategia_periodo.append(retorno_periodo)

    # Imprimir os resultados do período
    print(f"Ações compradas: {acoes_compradas['Simbolo'].tolist()}")
    print("Retorno de cada ação:")
    for index, row in acoes_compradas.iterrows():
        print(f" - {row['Simbolo']}: {row['Retorno Real']}%")
    print(f"Retorno do período: {retorno_periodo:.2f}%")
    print(f"Valor do portfólio ao final do período: ${valor_portfolio:.2f}\n")


Arquivo 'resultados_portfolio_mensal_full_modified.xlsx' lido com sucesso.

Processando a planilha: Período 1
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2019-02-22 - 2019-03-21
Selecionadas 5 ações para o portfólio.
Ações compradas: ['AMT', 'SO', 'MCD', 'WMT', 'XEL']
Retorno de cada ação:
 - AMT: 11.70644120100242%
 - SO: 4.33944426916547%
 - MCD: 2.262881295990945%
 - WMT: 1.500255889162151%
 - XEL: 4.488114776519038%
Retorno do período: 4.86%
Valor do portfólio ao final do período: $104859.43


Processando a planilha: Período 2
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2019-03-22 - 2019-04-18
Selecionadas 5 ações para o portfólio.
Ações compradas: ['WRB', 'SYY', 'WMT', 'L', 'YUM']
Retorno de cada ação:
 - WRB: 3.387195385279303%
 - SYY: 8.42385272730434%
 - WMT: 5.960145210909561%
 - L: 2.258000103301393%
 - YUM: 4.291298434993405%
Retorno do período: 4.86%
Valor do portfólio ao final do período: $109959.89


Processando a pl

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirs

Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2019-07-11 - 2019-05-12
Selecionadas 5 ações para o portfólio.
Ações compradas: ['VRSK', 'HSY', 'AWK', 'JNJ', 'AMT']
Retorno de cada ação:
 - VRSK: 5.583805182994976%
 - HSY: 5.38370653316616%
 - AWK: 6.7038027301709%
 - JNJ: 6.753337540317671%
 - AMT: 3.226699596781463%
Retorno do período: 5.53%
Valor do portfólio ao final do período: $170476.62


Processando a planilha: Período 11
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2019-06-12 - 2020-06-01
Selecionadas 5 ações para o portfólio.
Ações compradas: ['AMT', 'CNP', 'SO', 'L', 'ATO']
Retorno de cada ação:
 - AMT: 8.17041551267005%
 - CNP: 6.326813398384523%
 - SO: 3.433523775592092%
 - L: 4.095798046202806%
 - ATO: 3.44735424778083%
Retorno do período: 5.09%
Valor do portfólio ao final do período: $179162.03


Processando a planilha: Período 12
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2020-07-01 - 2020-0

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True

Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2021-03-17 - 2021-04-14
Selecionadas 5 ações para o portfólio.
Ações compradas: ['AWK', 'COST', 'AJG', 'XEL', 'O']
Retorno de cada ação:
 - AWK: 11.50741283507968%
 - COST: 8.226440785048307%
 - AJG: 8.576796622213587%
 - XEL: 7.44889056070354%
 - O: 4.905586762100404%
Retorno do período: 8.13%
Valor do portfólio ao final do período: $569147.31


Processando a planilha: Período 28
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2021-04-15 - 2021-12-05
Selecionadas 5 ações para o portfólio.
Ações compradas: ['MDLZ', 'AVB', 'L', 'AJG', 'WM']
Retorno de cada ação:
 - MDLZ: 3.892172885004371%
 - AVB: 0.0829864028408223%
 - L: 4.938152050934859%
 - AJG: 5.427566331083598%
 - WM: 2.424601180506462%
Retorno do período: 3.35%
Valor do portfólio ao final do período: $588231.37


Processando a planilha: Período 29
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2021-05-13 - 202

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirs

Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2022-07-22 - 2022-08-18
Selecionadas 5 ações para o portfólio.
Ações compradas: ['AFL', 'TJX', 'ALL', 'WRB', 'ECL']
Retorno de cada ação:
 - AFL: 14.20882830113481%
 - TJX: 11.67809866993204%
 - ALL: 12.84963822068831%
 - WRB: 6.743089112564267%
 - ECL: 8.778927127622849%
Retorno do período: 10.85%
Valor do portfólio ao final do período: $1305054.65


Processando a planilha: Período 45
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período: 2022-08-19 - 2022-09-16
Selecionadas 5 ações para o portfólio.
Ações compradas: ['WRB', 'ACGL', 'WMT', 'YUM', 'CNP']
Retorno de cada ação:
 - WRB: 0.9455726118680547%
 - ACGL: -0.3433476465357038%
 - WMT: -1.32758461789767%
 - YUM: 1.400640595682244%
 - CNP: -0.7152938855780259%
Retorno do período: -0.01%
Valor do portfólio ao final do período: $1304950.21


Processando a planilha: Período 46
Linhas processadas: 50 (removidas 0 linhas com dados ausentes)
Período:

C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:42: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_34352\791043141.py:41: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True

In [5]:
# Verificar se o histórico do portfólio está vazio
if not historico_portfolio or not datas_portfolio:
    print("Nenhum período foi processado com sucesso. Verifique os dados e tente novamente.")
    exit()
else:
    # Criar DataFrame com o histórico do portfólio
    df_portfolio = pd.DataFrame({
        'Data': datas_portfolio,
        'Valor': historico_portfolio
    })
    # Ordenar por data
    df_portfolio.sort_values(by='Data', inplace=True)
    df_portfolio.reset_index(drop=True, inplace=True)

    print("\nHistórico do Portfólio:")
    print(df_portfolio)



Histórico do Portfólio:
         Data         Valor
0  2019-03-21  1.048594e+05
1  2019-04-18  1.099599e+05
2  2019-05-12  1.704766e+05
3  2019-05-17  1.164842e+05
4  2019-06-11  1.615429e+05
5  2019-06-17  1.229856e+05
6  2019-07-16  1.291283e+05
7  2019-08-13  1.393745e+05
8  2019-09-10  1.527601e+05
9  2019-11-09  1.476409e+05
10 2020-04-02  1.922536e+05
11 2020-04-03  1.992954e+05
12 2020-04-30  2.235101e+05
13 2020-05-29  2.497388e+05
14 2020-06-01  1.791620e+05
15 2020-06-26  2.636060e+05
16 2020-07-27  2.859065e+05
17 2020-08-24  3.059696e+05
18 2020-09-22  3.118747e+05
19 2020-10-20  3.487903e+05
20 2020-11-17  3.866707e+05
21 2020-12-16  4.027791e+05
22 2021-01-11  7.966157e+05
23 2021-01-15  4.349880e+05
24 2021-02-16  4.792084e+05
25 2021-03-09  7.278980e+05
26 2021-03-16  5.263400e+05
27 2021-04-10  7.475197e+05
28 2021-04-14  5.691473e+05
29 2021-06-08  6.942125e+05
30 2021-09-07  6.690834e+05
31 2021-10-06  6.360894e+05
32 2021-11-30  7.976614e+05
33 2021-12-05  5.882314

In [6]:
# Obter dados do S&P500
data_inicio_total = min(datas_inicio_portfolio) - timedelta(days=5)
data_fim_total = max(datas_portfolio) + timedelta(days=5)
print(f"\nBaixando dados do S&P500 de {data_inicio_total.date()} a {data_fim_total.date()}.")
    
try:
    # Baixar dados do S&P500 sem o parâmetro 'group_by'
    sp500 = yf.download('^GSPC', start=data_inicio_total, end=data_fim_total)
    print("Dados do S&P500 baixados com sucesso.")
except Exception as e:
    print(f"Erro ao baixar dados do S&P500: {e}")
    exit()

# Resetar o índice para transformar o índice em coluna
sp500.reset_index(inplace=True)

# Verificar se as colunas são MultiIndex
if isinstance(sp500.columns, pd.MultiIndex):
    # Achatar as colunas combinando os níveis
    sp500.columns = [' '.join(col).strip() for col in sp500.columns.values]
    print("Ajustando colunas de MultiIndex para nível único.")
else:
    print("Colunas de sp500 são de nível único.")

# Converter os nomes das colunas para minúsculas para facilitar a comparação
sp500.columns = [col.lower() for col in sp500.columns]
print(f"Colunas em sp500 após conversão para minúsculas: {sp500.columns.tolist()}")

# Verificar o nome da coluna de data
date_column_name = None
for col in sp500.columns:
    if 'date' in col:
        date_column_name = col
        break

if date_column_name is None:
    print("Nenhuma coluna de data encontrada em sp500.")
    exit()

if date_column_name != 'date':
    sp500.rename(columns={date_column_name: 'date'}, inplace=True)
    print(f"Renomeada a coluna de data para 'date'.")

# Remover informações de fuso horário e normalizar a data
sp500['date'] = pd.to_datetime(sp500['date']).dt.tz_localize(None).dt.normalize()

# Determinar a coluna de preço a ser usada
price_column = None
for col in sp500.columns:
    if 'adj close' in col.lower():
        price_column = col
        break
if price_column is None:
    for col in sp500.columns:
        if 'close' in col.lower():
            price_column = col
            break

if price_column is None:
    raise KeyError(f"Nenhuma coluna de preço ('Adj Close' ou 'Close') encontrada em sp500. Colunas disponíveis: {sp500.columns.tolist()}")

print(f"Coluna de preço selecionada: '{price_column}'")

# Criar sp500_prices com 'date' como coluna
sp500_prices = sp500[['date', price_column]].rename(columns={price_column: 'Price'})

print(f"Colunas em sp500_prices: {sp500_prices.columns.tolist()}")

# Calcular Retorno do S&P500 (não acumulado)
sp500_prices['Retorno'] = sp500_prices['Price'].pct_change().fillna(0)

# Verificar se 'Retorno' foi criado
if 'Retorno' not in sp500_prices.columns:
    print("Erro: A coluna 'Retorno' não foi criada em sp500_prices.")
    print(f"Colunas disponíveis em sp500_prices: {sp500_prices.columns.tolist()}")
    exit()
else:
    print("'Retorno' criado com sucesso em sp500_prices.")
    print(sp500_prices[['date', 'Price', 'Retorno']].head())


[*********************100%***********************]  1 of 1 completed


Baixando dados do S&P500 de 2019-02-17 a 2023-12-31.
Dados do S&P500 baixados com sucesso.
Ajustando colunas de MultiIndex para nível único.
Colunas em sp500 após conversão para minúsculas: ['date', 'adj close ^gspc', 'close ^gspc', 'high ^gspc', 'low ^gspc', 'open ^gspc', 'volume ^gspc']
Coluna de preço selecionada: 'adj close ^gspc'
Colunas em sp500_prices: ['date', 'Price']
'Retorno' criado com sucesso em sp500_prices.
        date        Price   Retorno
0 2019-02-19  2779.760010  0.000000
1 2019-02-20  2784.699951  0.001777
2 2019-02-21  2774.879883 -0.003526
3 2019-02-22  2792.669922  0.006411
4 2019-02-25  2796.110107  0.001232


In [7]:
# Preparar required_dates
required_dates = pd.Series(datas_inicio_portfolio + datas_portfolio).drop_duplicates()
required_dates = pd.to_datetime(required_dates).dt.normalize()
required_dates = required_dates.to_frame(name='date')
required_dates.sort_values('date', inplace=True)
required_dates.reset_index(drop=True, inplace=True)

# Garantir que não há duplicatas em required_dates
required_dates.drop_duplicates(subset='date', inplace=True)

# Realizar o merge_asof para alinhar os preços do S&P500 com as datas do portfólio
prices_at_required_dates = pd.merge_asof(
    required_dates.sort_values('date'),
    sp500_prices.sort_values('date'),
    on='date',
    direction='backward'
)

# Verificar se há NaNs após o merge
na_prices = prices_at_required_dates['Price'].isna().sum()
if na_prices > 0:
    print(f"Atenção: {na_prices} datas no portfólio não têm correspondência no S&P500.")

# Criar um dicionário mapeando datas a preços
date_price_dict = dict(zip(prices_at_required_dates['date'], prices_at_required_dates['Price']))


In [8]:
# Calcular os retornos do S&P500 para cada período
for i in range(len(datas_inicio_portfolio)):
    data_inicio = datas_inicio_portfolio[i].normalize()
    data_fim = datas_portfolio[i].normalize()

    price_inicio = date_price_dict.get(data_inicio)
    price_fim = date_price_dict.get(data_fim)

    if pd.isna(price_inicio) or pd.isna(price_fim):
        print(f"Não foi possível obter preços para o período {data_inicio.date()} - {data_fim.date()}")
        retorno_sp500_periodo = np.nan
    else:
        retorno_sp500_periodo = (price_fim - price_inicio) / price_inicio * 100
        print(f"Retorno do S&P500 no período {data_inicio.date()} - {data_fim.date()}: {retorno_sp500_periodo:.2f}%")
    retornos_sp500_periodo.append(retorno_sp500_periodo)


Retorno do S&P500 no período 2019-02-22 - 2019-03-21: 2.23%
Retorno do S&P500 no período 2019-03-22 - 2019-04-18: 3.72%
Retorno do S&P500 no período 2019-04-22 - 2019-05-17: -1.67%
Retorno do S&P500 no período 2019-05-20 - 2019-06-17: 1.74%
Retorno do S&P500 no período 2019-06-18 - 2019-07-16: 2.96%
Retorno do S&P500 no período 2019-07-17 - 2019-08-13: -1.95%
Retorno do S&P500 no período 2019-08-14 - 2019-11-09: 8.89%
Retorno do S&P500 no período 2019-12-09 - 2019-09-10: -4.99%
Retorno do S&P500 no período 2019-10-10 - 2019-06-11: -1.78%
Retorno do S&P500 no período 2019-07-11 - 2019-05-12: -3.95%
Retorno do S&P500 no período 2019-06-12 - 2020-06-01: 6.11%
Retorno do S&P500 no período 2020-07-01 - 2020-04-02: -18.90%
Retorno do S&P500 no período 2020-05-02 - 2020-04-03: -12.08%
Retorno do S&P500 no período 2020-02-04 - 2020-04-30: -11.68%
Retorno do S&P500 no período 2020-01-05 - 2020-05-29: -5.89%
Retorno do S&P500 no período 2020-01-06 - 2020-06-26: -7.31%
Retorno do S&P500 no períod

In [9]:
# Criar DataFrame com os retornos por período
df_returns = pd.DataFrame({
    'Data_Inicio': datas_inicio_portfolio,
    'Data_Fim': datas_portfolio,
    'Retorno_Estrategia': retornos_estrategia_periodo,
    'Retorno_SP500': retornos_sp500_periodo
})

# Remover períodos onde o retorno do S&P500 é NaN
df_returns.dropna(subset=['Retorno_SP500'], inplace=True)

# Formatar as datas para serem usadas como rótulos no gráfico
df_returns['Período'] = df_returns['Data_Fim'].dt.strftime('%Y-%m-%d')

print("\nDataFrame com Retornos por Período:")
print(df_returns)



DataFrame com Retornos por Período:
   Data_Inicio   Data_Fim  Retorno_Estrategia  Retorno_SP500     Período
0   2019-02-22 2019-03-21            4.859427       2.227616  2019-03-21
1   2019-03-22 2019-04-18            4.864098       3.724772  2019-04-18
2   2019-04-22 2019-05-17            5.933391      -1.665765  2019-05-17
3   2019-05-20 2019-06-17            5.581319       1.740702  2019-06-17
4   2019-06-18 2019-07-16            4.994662       2.957417  2019-07-16
5   2019-07-17 2019-08-13            7.934921      -1.946772  2019-08-13
6   2019-08-14 2019-11-09            5.931065       8.888262  2019-11-09
7   2019-12-09 2019-09-10            3.467324      -4.992732  2019-09-10
8   2019-10-10 2019-06-11            5.749362      -1.783785  2019-06-11
9   2019-07-11 2019-05-12            5.530270      -3.950452  2019-05-12
10  2019-06-12 2020-06-01            5.094781       6.107627  2020-06-01
11  2020-07-01 2020-04-02            7.307123     -18.902011  2020-04-02
12  2020-05-02

In [10]:
# Plotar o gráfico de barras duplo dos retornos por período
fig_bar = go.Figure(data=[
    go.Bar(name='Estratégia', x=df_returns['Período'], y=df_returns['Retorno_Estrategia']),
    go.Bar(name='S&P500', x=df_returns['Período'], y=df_returns['Retorno_SP500'])
])

fig_bar.update_layout(
    barmode='group',
    title='Retornos por Período: Estratégia vs S&P500',
    xaxis_title='Período',
    yaxis_title='Retorno (%)',
    legend=dict(x=0.01, y=0.99)
)

fig_bar.show()


AJEITAR O GRÁFICO QUE FICOU RUIM(provavelmente por conta da escala)

# Exemplos

## Gráfico de barras verticais sobrepostas 

In [1]:
import plotly.graph_objects as go

# Dados de exemplo
x_values = ["Categoria A", "Categoria B", "Categoria C"]
y_values_1 = [10, 20, 15]
y_values_2 = [5, 15, 10]

# Criação das barras empilhadas
fig = go.Figure()

# Primeira barra (base)
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_1,
    name='Série 1'
))

# Segunda barra (sobreposta na anterior)
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_2,
    name='Série 2'
))

# Configuração do layout para barras empilhadas
fig.update_layout(
    barmode='stack',
    title='Gráfico de Barras Verticalmente Sobrepostas',
    xaxis_title='Categorias',
    yaxis_title='Valores'
)

# Exibe o gráfico
fig.show()


In [ ]:
def plot_time_serie_line_graph(serie: pd.Series, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Essa função é responsável por realizar o plot de uma série temporal.
        Args:
            serie (pd.Series): Série temporal cujos dados serão utilizados para o plot. Os valores de tal série temporal estarão no eixo y,
            enquanto os índices dessa série estarão no eixo x.
            title (string):
            xaxis_title (string):
            yaxis_title (string):
            
        Return:
            Essa função plota a série temporal, mas não retorna nada.
    '''
    
    # Cria a figure onde a série temporal em questão será plotada.
    fig = go.Figure()
    
    # Adiciona o gráfico da série temporal em questão ao plot.
    fig.add_trace(go.Scatter(
        # Define os valores do eixo x do plot.
        x = serie.index,
        # Define os valores do eixo y do plot.
        y = serie.values,
        # Define o tipo do gráfico que será plotado.
        mode = "lines"
    ))
    
    # Adiciona algumas legendas ao plot.
    fig.update_layout(
        # Adiciona um título ao plot.
        title = serie_title,
        # Adiciona uma legenda ao eixo x do plot.
        xaxis_title = serie_xaxis_title,
        # Adiciona uma legenda ao eixo y do plot.
        yaxis_title = serie_yaxis_title
    )
    
    # Exibe o plot criado.
    fig.show()

In [ ]:
'''
# Cria um título para o plot.
title = f"Retornos logarítmicos diários de {ticker1}"
# Cria uma legenda para o eixo x do plot.
xaxis_title = "Data"
# Cria uma legenda para o eixo y do plot.
yaxis_title = "Valor"

# Cria o plot da série temporal dos log retornos diários do "ticker1".
plot_time_serie_line_graph(ticker1_daily_logarithmic_return, title, xaxis_title, yaxis_title)
'''

In [ ]:
def plot_multiples_time_series_line_graphs(data_list: list, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Esta função exibe em um mesmo plot múltiplos gráficos de séries temporais. 
        Args:
            data_list (list): Lista contendo as séries temporais cujos gráficos serão exibidos no plot. Os valores de tais séries temporais
                              estarão no eixo y, enquanto os índices dessas séries estarão no eixo x.
            title (string) [Optional]: Título do plot.
            xaxis_title (string) [Optional]: Título do eixo x do plot.
            yaxis_title (string) [Optional]: Título do eixo y do plot.
        Return:
            None: A função exibe os gráficos, mas não retorna nenhum valor.  
        Errors:
            TypeError: É esperado que todas os elementos da lista "data_list" sejam objetos do tipo pd.Series, isto é, que sejam séries temporais.
            ValueError: É esperado que todas as séries temporais presentes na variável "data_list" possuam os mesmos índices. 
            TypeError: É esperado que todas as séries temporais presentes na variável data_list possuam um atributo "name".
    '''    

    # Verifica se todos os elementos presentes na lista "data_list" são séries temporais.
    are_all_data_time_series = all(isinstance(df, pd.Series) for df in data_list)
    
    # Retorna um erro caso algum dos dados presentes na variável "data_list" não seja uma série temporal
    if not are_all_data_time_series:
        raise TypeError("Todos os dados presentes no parâmetro 'data_list' devem ser séries temporais.")
    
    # Verifica se as séries temporais presentes na variável "data_list" possuem os mesmos índices.
    are_all_index_equal = all(df.index.equals(data_list[0].index) for df in data_list)
    
    # Retorna um erro caso as séries temporais possuam índices diferentes.
    if not are_all_index_equal:
        raise ValueError("Todos as séries temporais devem possuir os mesmos índices.")
    
    # Verifica se todas as séries temporais presentes na variável "data_list" possuem o atributo "name".
    all_time_series_have_names = all(hasattr(df,"name") for df in data_list)
    
    # Retorna um erro caso uma das séries temporais presentes na variável "data_list" não possua o atributo "name".
    if not all_time_series_have_names:
        raise TypeError("Todas as séries temporais devem possuir o atributo 'name'")
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data_list[0].index.tolist() # Observe que só podemos fazer isso pois temos certeza que todas as séries temporais possuem os mesmos índices.
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for time_serie in data_list:
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=time_serie.values, mode="lines", name=time_serie.name))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        # Seta um título para o plot.
        title=serie_title,
        # Seta um título para o eixo x do plot.
        xaxis_title=serie_xaxis_title,
        # Seta um título para o eixo y do plot.
        yaxis_title=serie_yaxis_title,   
    )
    
    # Exibe o gráfico criado.
    fig.show()

#

In [7]:
setup = {
    #
    "monthly_results_sheets": ['resultados_portfolio_mensal_full_1.xlsx', 'resultados_portfolio_mensal_full_2.xlsx',
                               'resultados_portfolio_mensal_full_3.xlsx', 'resultados_portfolio_mensal_full_4.xlsx'],
    #
    "weekly_results_sheets": ['resultados_portfolio_semanal_full_1.xlsx']
}

In [15]:
def process_file_and_get_results(filename: str) -> pd.DataFrame:
    '''
        Description:

        Args:

        Return:
    '''

    # Ler todas as planilhas do arquivo Excel
    dfs = pd.read_excel(filename, sheet_name=None)
    
    #
    first_trading_day = dfs['Período 1'][dfs['Período 1'].columns[0]][0].split(" - ")[0]
    #
    first_trading_day = np.datetime64(first_trading_day)
    
    #
    results = pd.DataFrame({
        "Retorno": 0
    }, index=[first_trading_day])
    
    # Acessar e trabalhar com cada DataFrame
    for period, df in dfs.items():
        if not df.empty:
            #
            period_last_trading_day = df[df.columns[0]][0].split(" - ")[1]
            #
            period_last_trading_day = np.datetime64(period_last_trading_day)
            #
            tickers_to_trade = math.ceil(df.shape[0]*0.1)
            #
            period_return = df.iloc[:tickers_to_trade]['Retorno Real'].mean() # Trocar por um .sum() ?
            #
            result = pd.DataFrame({
                "Retorno": period_return,
            },index=[period_last_trading_day])
            #
            results = pd.concat([results, result])
    
    return results
            

In [16]:
process_file_and_get_results(setup['weekly_results_sheets'][0])

,Retorno
2019-01-31,0.000000
2019-02-06,-0.119412
2019-02-13,-0.023905
2019-02-21,0.110191
2019-02-28,0.447097
...,...
2023-11-27,-0.243826
2023-12-04,0.677020
2023-12-11,0.474322
2023-12-18,-0.061220


## **Sugestões**

**Gŕaficos a serem feitos:** *(Ver em quais gráficos deve ser plotada uma região de confiança)*

    - Gráfico de série temporal com o maior retorno acumulado, menor retorno acumulado e retorno médio acumulado da estratégia.
  
    - Gráfico de série temporal com o retorno médio acumulado da estratégia (logarítmico) x retorno médio acumulado do S&P500 (logarítmico).
  
    - Gráfico de série temporal com o retorno médio por período da estratégia x sharpe ratio por período da estratégia x sortino ratio por  período da estratégia.
  
    - Histograma com a distribuição dos retornos médios por período da estratégia *(plotar sobre os histogramas uma linha de distribuição normal ajustada)*. Para esse histograma, é importante calcular a assimetria (quanto mais próximo de 0, mais "normal" é a distribuição) e a kurtosis (quanto mais próximo de 3, mais "normal" é a distribuição).
  
    - Gráfico de barras com os retornos médios por período da estratégia x retornos por período do S&P 500.